In [6]:
import requests
import pandas as pd

def generate_points(latitude, longitude, distance=0.1):
    # Generates 8 points around the original location in a roughly square grid
    print("Generating points around the original coordinates...")
    return [
        (latitude + distance, longitude),  # North
        (latitude - distance, longitude),  # South
        (latitude, longitude + distance),  # East
        (latitude, longitude - distance),  # West
        (latitude + distance, longitude + distance),  # Northeast
        (latitude - distance, longitude - distance),  # Southwest
        (latitude + distance, longitude - distance),  # Northwest
        (latitude - distance, longitude + distance),  # Southeast
    ]

def fetch_data(latitude, longitude, offset=0):
    print(f"Fetching data for latitude: {latitude}, longitude: {longitude}, offset: {offset}")
    url = f"https://api.yelp.com/v3/businesses/search?latitude={latitude}&longitude={longitude}&radius=40000&categories=restaurants,food&price=1,2,3,4&sort_by=distance&limit=50&offset={offset}"
    headers = {
        "Authorization": "youapikeyhere"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Stadium coordinates
original_latitude = 33.7548
original_longitude = -84.40204
points = generate_points(original_latitude, original_longitude)

unique_businesses = {}

for idx, point in enumerate(points):
    print(f"Processing point {idx + 1}/{len(points)}...")
    offset = 0
    while True:
        data = fetch_data(point[0], point[1], offset)
        if not data or 'businesses' not in data:
            print("No more data or failed to fetch data. Moving to next point.")
            break  # Stop if no data returned or finished all businesses for this point
        
        for business in data['businesses']:
            if business['id'] not in unique_businesses:
                categories = ', '.join([cat['title'] for cat in business['categories']])
                address = ', '.join(business['location']['display_address'])
                unique_businesses[business['id']] = {
                    'ID': business['id'],
                    'Name': business['name'],
                    'Image URL': business['image_url'],
                    'URL': business['url'],
                    'Is Closed': business['is_closed'],
                    'Review Count': business['review_count'],
                    'Categories': categories,
                    'Rating': business['rating'],
                    'Latitude': business['coordinates']['latitude'],
                    'Longitude': business['coordinates']['longitude'],
                    'Transactions': ', '.join(business['transactions']),
                    'Price': business.get('price', ''),  # Some businesses might not have a price
                    'Address': address,
                    'Phone': business['phone'],
                    'Display Phone': business['display_phone'],
                    'Distance': business['distance']
                }
                print(f"Added unique business: {business['name']} (ID: {business['id']})")
        offset += 50

# Convert dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(unique_businesses, orient='index')

# Save the DataFrame to an Excel file
excel_file_path = 'C:/Users/agutierrez/Downloads/AtlantaYelpRestaurants.xlsx'
df.to_excel(excel_file_path, index=False)

print(f"Yelp restaurant data saved to '{excel_file_path}'")

Generating points around the original coordinates...
Processing point 1/8...
Fetching data for latitude: 33.854800000000004, longitude: -84.40204, offset: 0
Added unique business: Walgreens (ID: 52AVRBhYF5Ag2YmRpQ59HQ)
Added unique business: Landmark Diner (ID: XzWLpzKHQLnJiNM_INxBag)
Added unique business: Dunkin' (ID: alYvRpB1aJXQnKJqhAUsaA)
Added unique business: Jersey Mike's Subs (ID: xijb3kWsaOtj7AMzhq9cQw)
Added unique business: JINYA Ramen Bar - Buckhead (ID: 57d0Gmx4JKuKQWyJYu0yfg)
Added unique business: Panera Bread (ID: IXi0TYqFJJUIzUaUs_OX1A)
Added unique business: Postino Buckhead (ID: TDvnkXWrEj4vBfeAgDDiRQ)
Added unique business: CAVA (ID: LDdeNnXmP6wgaqummfCe8w)
Added unique business: The Ivy (ID: mUYx6CkbQLp3KCjF5LfSIw)
Added unique business: Tin Lizzy's Cantina (ID: Tr-8sVrdh9tfAsW7LaZ_Vg)
Added unique business: Edible Arrangements (ID: 2Qv3MyW87rhfYS-MxbRYMA)
Added unique business: Chopt Creative Salad Co. (ID: UQ0gqyDzvIMbCKpwxQJxOg)
Added unique business: Menchie's

In [4]:
import folium

# Original coordinates
original_latitude = 33.7548
original_longitude = -84.40204

# Function to generate 8 surrounding points
def generate_points(latitude, longitude, distance=0.1):
    return [
        (latitude + distance, longitude),  # North
        (latitude - distance, longitude),  # South
        (latitude, longitude + distance),  # East
        (latitude, longitude - distance),  # West
        (latitude + distance, longitude + distance),  # Northeast
        (latitude - distance, longitude - distance),  # Southwest
        (latitude + distance, longitude - distance),  # Northwest
        (latitude - distance, longitude + distance),  # Southeast
    ]

# Generating points
points = generate_points(original_latitude, original_longitude)

# Create a map centered at the original coordinates
map_ = folium.Map(location=[original_latitude, original_longitude], zoom_start=12)

# Mark the original location
folium.Marker([original_latitude, original_longitude], popup='Original Location', icon=folium.Icon(color='red')).add_to(map_)

# Add markers for the 8 surrounding points
for idx, (lat, lon) in enumerate(points):
    folium.Marker([lat, lon], popup=f'Point {idx+1}', icon=folium.Icon(color='blue')).add_to(map_)

# Display the map
map_
